0. Start
1. Stop
2. Change Lane
3. Turn left
4. Turn Right
5. Speed Up
6. Speed Down
7. Chalo
8. Ruk jao
9. Lane badlo
10. Bayain Muro
11. Dayain Muro
12. Tez chalo
13. Ahista Chalo

<h1>In this notebook models are trained on newly generated data set where each command is ovelayed
with 5 different background noises</h1>

In [25]:
import librosa
import os
audioLength=2.5
filesBasicPath='VoiceData/multipleBackground/'
numberOfCommands=85
audioFiles=[]
foldersList=['start','stop','changeLane','turnLeft','turnRight','speedUp','speedDown','chalo','rukJao','laneBadlo','bayainMuro','dayainMuro','tezChalo','ahistaChalo']
outputs=[]
path = os.getcwd()
count=0
for i in range(0,len(foldersList)):
    audioFiles.extend(os.listdir(path+'/'+filesBasicPath+foldersList[i]))
    for j in range(count,len(audioFiles)):
        count+=1
        outputs.append(i)
        audioFiles[j]=filesBasicPath+foldersList[i]+'/'+audioFiles[j]

print(len(audioFiles))
print(len(outputs))
print("File Loading is completed")


5430
5430
File Loading is completed


In [26]:
timeSeriesValuesList=[]
samplingRateList=[]
for i in range(0,len(audioFiles)):
    timeSeriesValue,samplingRate=librosa.load(audioFiles[i])
    timeSeriesValuesList.append(timeSeriesValue)
    samplingRateList.append(samplingRate)

print("Time series and sampling rate extraction completed")

Time series and sampling rate extraction completed


In [27]:
print(len(samplingRateList))
print(len(timeSeriesValuesList))

5430
5430


In [28]:
print(samplingRateList[0])
print(samplingRateList[1])

22050
22050


In [29]:
import numpy as np
mfccs=[]
for i in range(0,len(audioFiles)):
    mfccs.append(librosa.feature.mfcc(timeSeriesValuesList[i], n_mfcc=13,sr=samplingRateList[i]).T)
    
print("Mfcc extraction completed")

Mfcc extraction completed


np.save("savedMfccs/mfccs_5background.npy",np.array(mfccs))

mfccs_loaded=np.load("savedMfccs/mfccs_5background.npy")
print(np.shape(np.array(mfccs_loaded)))


In [30]:
mfccs2=np.array(mfccs)

In [38]:
print(np.shape(np.array(mfccs)))
print(type(mfccs))
print(type(mfccs[0]))

(5430, 108, 13)
<class 'list'>
<class 'numpy.ndarray'>


In [39]:
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mfccs, outputs, test_size=0.25, random_state=42)
nb_classes = 14
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print("Train Test Split Finished")

Train Test Split Finished


In [43]:
np.save("trainTestSplitLists/X_train_25_5background.npy",np.array(X_train))
np.save("trainTestSplitLists/Y_train_25_5background.npy",np.array(Y_train))
np.save("trainTestSplitLists/X_test_25_5background.npy",np.array(X_test))
np.save("trainTestSplitLists/Y_test_25_5background.npy",np.array(Y_test))

In [44]:
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y_test))

(4072, 108, 13)
(4072, 14)
(1358, 108, 13)
(1358, 14)


In [45]:
from tensorflow.keras.optimizers import SGD
np.random.seed(1337)
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from tensorflow.keras.optimizers import Adam
from keras.layers import Bidirectional

modelLSTM = Sequential()
input_shape = (np.array(X_train).shape[1], np.array(X_train).shape[2])
modelLSTM.add(LSTM(units=108, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
modelLSTM.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
modelLSTM.add(Dense(units=Y_train.shape[1], activation="softmax"))
opt = Adam()
modelLSTM.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
modelLSTM.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 108, 108)          52704     
                                                                 
 lstm_1 (LSTM)               (None, 32)                18048     
                                                                 
 dense (Dense)               (None, 14)                462       
                                                                 
Total params: 71,214
Trainable params: 71,214
Non-trainable params: 0
_________________________________________________________________


In [46]:
# print("Training ...")
batch_size = 407   # num of training examples per minibatch
num_epochs = 50
# print(len(x_train))
# print(len(Y_train))
modelLSTM.fit(
    np.array(X_train),
    np.array(Y_train),
    batch_size=batch_size,
    epochs=num_epochs,
)

Epoch 1/50
11/11 [==============================] - 23s 2s/step - loss: 2.6584 - accuracy: 0.0778
Epoch 2/50
11/11 [==============================] - 20s 2s/step - loss: 2.6463 - accuracy: 0.0739
Epoch 3/50
11/11 [==============================] - 20s 2s/step - loss: 2.6410 - accuracy: 0.0793
Epoch 4/50
11/11 [==============================] - 22s 2s/step - loss: 2.6439 - accuracy: 0.0786
Epoch 5/50
11/11 [==============================] - 22s 2s/step - loss: 2.6419 - accuracy: 0.0751
Epoch 6/50
11/11 [==============================] - 22s 2s/step - loss: 2.6399 - accuracy: 0.0781
Epoch 7/50
11/11 [==============================] - 22s 2s/step - loss: 2.6382 - accuracy: 0.0781
Epoch 8/50
11/11 [==============================] - 23s 2s/step - loss: 2.6355 - accuracy: 0.0850
Epoch 9/50
11/11 [==============================] - 22s 2s/step - loss: 2.6401 - accuracy: 0.0761
Epoch 10/50
11/11 [==============================] - 23s 2s/step - loss: 2.6418 - accuracy: 0.0791
Epoch 11/50
11/11 [

In [48]:
print("\nTesting LSTM...")
score, accuracy = modelLSTM.evaluate(
    np.array(X_test), np.array(Y_test), batch_size=batch_size, verbose=1)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)


Testing LSTM...
4/4 [==============================] - 1s 154ms/step - loss: 2.2950 - accuracy: 0.1863
Test loss:   2.29496693611145
Test accuracy:   0.18630339205265045


<h1>BiLSTM model development, training and testing in the following code</h1>

In [50]:
modelBiLSTM = Sequential()
input_shape = (np.array(X_train).shape[1], np.array(X_train).shape[2])
modelBiLSTM.add(Bidirectional(LSTM(units=108, dropout=0.05, recurrent_dropout=0.35, return_sequences=True), input_shape=input_shape))
modelBiLSTM.add(Bidirectional(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False)))
modelBiLSTM.add(Dense(units=Y_train.shape[1], activation="softmax"))
opt = Adam()
modelBiLSTM.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
modelBiLSTM.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 108, 216)         105408    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               63744     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 14)                910       
                                                                 
Total params: 170,062
Trainable params: 170,062
Non-trainable params: 0
_________________________________________________________________


In [53]:
# print("Training ...")
batch_size = 407   # num of training examples per minibatch
num_epochs = 100
# print(len(x_train))
# print(len(Y_train))
modelBiLSTM.fit(
    np.array(X_train),
    np.array(Y_train),
    batch_size=batch_size,
    epochs=num_epochs,
)

Epoch 1/100
11/11 [==============================] - 138s 13s/step - loss: 2.1594 - accuracy: 0.2471
Epoch 2/100
11/11 [==============================] - 137s 12s/step - loss: 2.1160 - accuracy: 0.2750
Epoch 3/100
11/11 [==============================] - 135s 12s/step - loss: 2.1192 - accuracy: 0.2586
Epoch 4/100
11/11 [==============================] - 138s 13s/step - loss: 2.0940 - accuracy: 0.2647
Epoch 5/100
11/11 [==============================] - 142s 13s/step - loss: 2.0624 - accuracy: 0.2647
Epoch 6/100
11/11 [==============================] - 127s 11s/step - loss: 2.0227 - accuracy: 0.2895
Epoch 7/100
11/11 [==============================] - 128s 11s/step - loss: 2.0049 - accuracy: 0.3001
Epoch 8/100
11/11 [==============================] - 127s 11s/step - loss: 2.0133 - accuracy: 0.2937
Epoch 9/100
11/11 [==============================] - 128s 11s/step - loss: 1.9901 - accuracy: 0.2890
Epoch 10/100
11/11 [==============================] - 129s 12s/step - loss: 1.9133 - accura

11/11 [==============================] - 93s 8s/step - loss: 0.9026 - accuracy: 0.6702
Epoch 83/100
11/11 [==============================] - 93s 8s/step - loss: 0.9522 - accuracy: 0.6508
Epoch 84/100
11/11 [==============================] - 91s 8s/step - loss: 0.9034 - accuracy: 0.6591
Epoch 85/100
11/11 [==============================] - 91s 8s/step - loss: 0.8930 - accuracy: 0.6579
Epoch 86/100
11/11 [==============================] - 91s 8s/step - loss: 0.8883 - accuracy: 0.6618
Epoch 87/100
11/11 [==============================] - 91s 8s/step - loss: 0.8888 - accuracy: 0.6606
Epoch 88/100
11/11 [==============================] - 91s 8s/step - loss: 0.9197 - accuracy: 0.6574
Epoch 89/100
11/11 [==============================] - 90s 8s/step - loss: 1.0441 - accuracy: 0.6186
Epoch 90/100
11/11 [==============================] - 91s 8s/step - loss: 1.0225 - accuracy: 0.6272
Epoch 91/100
11/11 [==============================] - 90s 8s/step - loss: 0.9697 - accuracy: 0.6444
Epoch 92/100


In [54]:
print("\nTesting BiLstm...")
score, accuracy = modelBiLSTM.evaluate(
    np.array(X_test), np.array(Y_test), batch_size=batch_size, verbose=1)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)


Testing BiLstm...
4/4 [==============================] - 5s 1s/step - loss: 1.0526 - accuracy: 0.6296
Test loss:   1.0526479482650757
Test accuracy:   0.6296023726463318


In [ ]:
modelFileName="biLstm5_120_407.h5"
modelBiLSTM.save("savedModels/"+modelFileName)